### SECTION 1: ENVIRONMENT SETUP

In [1]:
# Install necessary libraries
!pip install pymysql
!pip install mysql-connector-python
!pip install sqlalchemy
!pip install pymongo
!pip install pandas
!pip install matplotlib

In [2]:
# Import libraries
import os
import pymysql
import mysql.connector
from sqlalchemy import create_engine
import pymongo
import datetime
import json
import requests
import string
import random

import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

### SECTION 2: CONNECT TO THE ORDER_MANAGEMENT DATABASE

In [3]:
# Database connection configuration
host = "localhost"
user = "root"
password = "ds2002spring2024"
main_database = "order_management"
extract_database = "northwind"

# Create an SQLAlchemy engine for the order_management database
order_management_engine = create_engine(f'mysql+pymysql://{user}:{password}@{host}/{main_database}')

# Connect to the order_management database
try:
    order_management_connection = pymysql.connect(host=host, user=user, password=password, database=main_database)
    print("Connection to the order_management database was successful!")
except Exception as e:
    print(f"Error connecting to the order_management database: {e}")

Connection to the order_management database was successful!


### SECTION 3: EXTRACT DATA FROM THE NORTHWIND MYSQL DATABASE AND LOAD DATA INTO THE ORDER_MANAGEMENT DATABASE

In [4]:
# Create an SQLAlchemy engine for the northwind database
northwind_engine = create_engine(f'mysql+pymysql://{user}:{password}@{host}/{extract_database}')

# Connect to the northwind database
try:
    northwind_connection = pymysql.connect(host=host, user=user, password=password, database=extract_database)
    print("Connection to the northwind database was successful!")
except Exception as e:
    print(f"Error connecting to the northwind database: {e}")

Connection to the northwind database was successful!


In [5]:
# Extract customers data from the northwind database
try:
    northwind_customers_query = "SELECT id, first_name, last_name, email_address, business_phone, address FROM customers"
    northwind_customers_df = pd.read_sql(northwind_customers_query, northwind_connection)
except Exception as e:
    print(f"Error extracting data from the Northwind database: {e}")

# Modify the columns from the northwind database to fit the order_management database
try:
    northwind_customers_df.rename(columns={
        'id': 'CustomerID',
        'first_name': 'FirstName',
        'last_name': 'LastName',
        'email_address': 'Email',
        'business_phone': 'Phone',
        'address': 'Address'
    }, inplace=True)
except Exception as e:
    print(f"Error modifying data: {e}")

northwind_customers_df.head()

,CustomerID,FirstName,LastName,Email,Phone,Address
0,1,Anna,Bedecs,None,(123)555-0100,123 1st Street
1,2,Antonio,Gratacos Solsona,None,(123)555-0100,123 2nd Street
2,3,Thomas,Axen,None,(123)555-0100,123 3rd Street
3,4,Christina,Lee,None,(123)555-0100,123 4th Street
4,5,Martin,O’Donnell,None,(123)555-0100,123 5th Street


In [6]:
# Load the customers data into the order_management database
try:
    northwind_customers_df.to_sql('Customers', order_management_engine, if_exists='append', index=False)
except Exception as e:
    print(f"Error loading data into the order_management database: {e}")

In [7]:
# Extract products data from the northwind database
try:
    northwind_products_query = "SELECT id, product_name, category, list_price, minimum_reorder_quantity FROM products"
    northwind_products_df = pd.read_sql(northwind_products_query, northwind_connection)
except Exception as e:
    print(f"Error extracting data from the Northwind database: {e}")

# Modify the columns from the northwind database to fit the order_management database
try:
    northwind_products_df.rename(columns={
        'id': 'ProductID',
        'product_name': 'Name',
        'category': 'Description',
        'list_price': 'Price',
        'minimum_reorder_quantity': 'StockQuantity'
    }, inplace=True)
except Exception as e:
    print(f"Error modifying data: {e}")

northwind_products_df.head()

,ProductID,Name,Description,Price,StockQuantity
0,1,Northwind Traders Chai,Beverages,18.00,10.0
1,3,Northwind Traders Syrup,Condiments,10.00,25.0
2,4,Northwind Traders Cajun Seasoning,Condiments,22.00,10.0
3,5,Northwind Traders Olive Oil,Oil,21.35,10.0
4,6,Northwind Traders Boysenberry Spread,"Jams, Preserves",25.00,25.0


In [8]:
# Load the products data into the order_management database
try:
    northwind_products_df.to_sql('Products', order_management_engine, if_exists='append', index=False)
except Exception as e:
    print(f"Error loading data into the order_management database: {e}")

In [9]:
# Extract orders data from the northwind database
try:
    northwind_orders_query = "SELECT id, customer_id, order_date, ship_address, status_id FROM orders"
    northwind_orders_df = pd.read_sql(northwind_orders_query, northwind_connection)
except Exception as e:
    print(f"Error extracting data from the Northwind database: {e}")

# Modify the columns from the northwind database to fit the order_management database
try:
    northwind_orders_df.rename(columns={
        'id': 'OrderID',
        'customer_id': 'CustomerID',
        'order_date': 'OrderDate',
        'ship_address': 'ShippingAddress',
        'status_id': 'Status'
    }, inplace=True)
    status_mapping = {
        0: 'In Process',
        1: 'On Hold',
        2: 'Disputed',
        3: 'Shipped'
    }
    northwind_orders_df['Status'] = northwind_orders_df['Status'].replace(status_mapping)
except Exception as e:
    print(f"Error modifying data: {e}")

northwind_orders_df.head()

,OrderID,CustomerID,OrderDate,ShippingAddress,Status
0,30,27,2006-01-15,789 27th Street,Shipped
1,31,4,2006-01-20,123 4th Street,Shipped
2,32,12,2006-01-22,123 12th Street,Shipped
3,33,8,2006-01-30,123 8th Street,Shipped
4,34,4,2006-02-06,123 4th Street,Shipped


In [10]:
# Load the orders data into the order_management database
try:
    northwind_orders_df.to_sql('Orders', order_management_engine, if_exists='append', index=False)
except Exception as e:
    print(f"Error loading data into the order_management database: {e}")

In [11]:
# Extract order details (items) data from the northwind database
try:
    northwind_order_items_query = "SELECT id, order_id, product_id, quantity, unit_price FROM order_details"
    northwind_order_items_df = pd.read_sql(northwind_order_items_query, northwind_connection)
except Exception as e:
    print(f"Error extracting data from the Northwind database: {e}")

# Modify the columns from the northwind database to fit the order_management database
try:
    northwind_order_items_df.rename(columns={
        'id': 'OrderItemID',
        'order_id': 'OrderID',
        'product_id': 'ProductID',
        'quantity': 'Quantity',
        'unit_price': 'Price'
    }, inplace=True)
except Exception as e:
    print(f"Error modifying data: {e}")

northwind_order_items_df.head()

,OrderItemID,OrderID,ProductID,Quantity,Price
0,27,30,34,100.0,14.0
1,28,30,80,30.0,3.5
2,29,31,7,10.0,30.0
3,30,31,51,10.0,53.0
4,31,31,80,10.0,3.5


In [12]:
# Load the order details (items) data into the order_management database
try:
    northwind_order_items_df.to_sql('OrderItems', order_management_engine, if_exists='append', index=False)
except Exception as e:
    print(f"Error loading data into the order_management database: {e}")

In [13]:
# Extract suppliers data from the northwind database
try:
    northwind_suppliers_query = "SELECT id, first_name, last_name, business_phone, address, company FROM suppliers"
    northwind_suppliers_df = pd.read_sql(northwind_suppliers_query, northwind_connection)
except Exception as e:
    print(f"Error extracting data from the Northwind database: {e}")

# Modify the columns from the northwind database to fit the order_management database
try:
    northwind_suppliers_df.rename(columns={
        'id': 'SupplierID',
        'first_name': 'FirstName',
        'last_name': 'LastName',
        'business_phone': 'Phone',
        'address': 'Address',
        'company': 'Company'
    }, inplace=True)
except Exception as e:
    print(f"Error modifying data: {e}")

northwind_suppliers_df.head()

,SupplierID,FirstName,LastName,Phone,Address,Company
0,1,Elizabeth A.,Andersen,None,None,Supplier A
1,2,Cornelia,Weiler,None,None,Supplier B
2,3,Madeleine,Kelley,None,None,Supplier C
3,4,Naoki,Sato,None,None,Supplier D
4,5,Amaya,Hernandez-Echevarria,None,None,Supplier E


In [14]:
# Load the suppliers data into the order_management database
try:
    northwind_suppliers_df.to_sql('Suppliers', order_management_engine, if_exists='append', index=False)
except Exception as e:
    print(f"Error loading data into the order_management database: {e}")

### SECTION 4: EXTRACT DATA FROM A CSV FILE AND LOAD DATA INTO THE ORDER_MANAGEMENT DATABASE

In [15]:
# Load the CSV file into a DataFrame
csv_sales_data_df = pd.read_csv('sales_data_sample.csv', encoding='ISO-8859-1')
csv_sales_data_df.head()

,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,...,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE
0,10107,30,95.70,2,2871.00,2/24/2003 0:00,Shipped,1,2,2003,...,897 Long Airport Avenue,NaN,NYC,NY,10022,USA,NaN,Yu,Kwai,Small
1,10121,34,81.35,5,2765.90,5/7/2003 0:00,Shipped,2,5,2003,...,59 rue de l'Abbaye,NaN,Reims,NaN,51100,France,EMEA,Henriot,Paul,Small
2,10134,41,94.74,2,3884.34,7/1/2003 0:00,Shipped,3,7,2003,...,27 rue du Colonel Pierre Avia,NaN,Paris,NaN,75508,France,EMEA,Da Cunha,Daniel,Medium
3,10145,45,83.26,6,3746.70,8/25/2003 0:00,Shipped,3,8,2003,...,78934 Hillside Dr.,NaN,Pasadena,CA,90003,USA,NaN,Young,Julie,Medium
4,10159,49,100.00,14,5205.27,10/10/2003 0:00,Shipped,4,10,2003,...,7734 Strong St.,NaN,San Francisco,CA,NaN,USA,NaN,Brown,Julie,Medium


In [16]:
# Extract and transform customers data from the CSV file
try:
    csv_customers_df = csv_sales_data_df[['CUSTOMERNAME', 'PHONE', 'ADDRESSLINE1']].copy()
    csv_customers_df.drop_duplicates(inplace=True)
    # Modify the columns from the CSV file to fit the order_management database
    csv_customers_df.rename(columns={
        'CUSTOMERNAME': 'FirstName',
        'PHONE': 'Phone',
        'ADDRESSLINE1': 'Address'
    }, inplace=True)
    csv_customers_df.insert(1, 'LastName', '')
    csv_customers_df.insert(2, 'Email', '')
except Exception as e:
    print(f"Error extracting and transforming customer data: {e}")

csv_customers_df.head()

,FirstName,LastName,Email,Phone,Address
0,Land of Toys Inc.,,,2125557818,897 Long Airport Avenue
1,Reims Collectables,,,26.47.1555,59 rue de l'Abbaye
2,Lyon Souveniers,,,+33 1 46 62 7555,27 rue du Colonel Pierre Avia
3,Toys4GrownUps.com,,,6265557265,78934 Hillside Dr.
4,Corporate Gift Ideas Co.,,,6505551386,7734 Strong St.


In [17]:
# Load the customers data into the order_management database
try:
    csv_customers_df.to_sql('Customers', order_management_engine, if_exists='append', index=False)
except Exception as e:
    print(f"Error loading data into the order_management database: {e}")

In [18]:
# Extract and transform products data from the CSV file
try:
    csv_products_df = csv_sales_data_df[['PRODUCTLINE', 'DEALSIZE', 'MSRP', 'QUANTITYORDERED']].copy()
    csv_products_df.drop_duplicates(inplace=True)
    # Modify the columns from the CSV file to fit the order_management database
    csv_products_df.rename(columns={
        'PRODUCTLINE': 'Name',
        'DEALSIZE': 'Description',
        'MSRP': 'Price',
        'QUANTITYORDERED': 'StockQuantity'
    }, inplace=True)
except Exception as e:
    print(f"Error extracting and transforming product data: {e}")

csv_products_df.head()

,Name,Description,Price,StockQuantity
0,Motorcycles,Small,95,30
1,Motorcycles,Small,95,34
2,Motorcycles,Medium,95,41
3,Motorcycles,Medium,95,45
4,Motorcycles,Medium,95,49


In [19]:
# Load the products data into the order_management database
try:
    csv_products_df.to_sql('Products', order_management_engine, if_exists='append', index=False)
except Exception as e:
    print(f"Error loading data into the order_management database: {e}")

In [20]:
# Extract and transform orders data from the CSV file
try:
    csv_orders_df = csv_sales_data_df[['ORDERNUMBER', 'ORDERDATE', 'ADDRESSLINE2', 'STATUS']].copy()
    csv_orders_df.drop_duplicates(inplace=True)
    # Modify the columns from the CSV file to fit the order_management database
    csv_orders_df.rename(columns={
        'ORDERNUMBER': 'OrderID',
        'ORDERDATE': 'OrderDate',
        'ADDRESSLINE2': 'ShippingAddress',
        'STATUS': 'Status'
    }, inplace=True)
    # Convert the orderdate column to datetime format
    csv_orders_df['OrderDate'] = pd.to_datetime(csv_orders_df['OrderDate'], format='%m/%d/%Y %H:%M')
except Exception as e:
    print(f"Error extracting and transforming order data: {e}")

csv_orders_df.head()

,OrderID,OrderDate,ShippingAddress,Status
0,10107,2003-02-24,NaN,Shipped
1,10121,2003-05-07,NaN,Shipped
2,10134,2003-07-01,NaN,Shipped
3,10145,2003-08-25,NaN,Shipped
4,10159,2003-10-10,NaN,Shipped


In [21]:
# Get the customerid for each order by merging dataframes
try:
    # Merge the csv_orders_df with csv_sales_data_df on the orderid and ordernumber columns
    csv_orders_merged_df = pd.merge(csv_orders_df, csv_sales_data_df, how='left', left_on='OrderID', right_on='ORDERNUMBER')
    csv_orders_merged_df.drop_duplicates(inplace=True)
    csv_orders_merged_df.drop('ORDERNUMBER', axis=1, inplace=True)
    # Extract the orderid and customername columns from csv_orders_merged_df
    order_customer_df = csv_orders_merged_df[['OrderID', 'CUSTOMERNAME']].copy()
    # Merge the order_customer_df with csv_customers_df on the firstname column
    customers_merged_df = pd.merge(order_customer_df, csv_customers_df[['FirstName']], how='left', left_on='CUSTOMERNAME', right_on='FirstName')
    customers_merged_df.drop('CUSTOMERNAME', axis=1, inplace=True)
    customers_merged_df.drop_duplicates(inplace=True)
except Exception as e:
    print(f"Error in merging and processing dataframes: {e}")

try:
    # SQL query to fetch customerid and firstname from customers table
    customer_id_query = "SELECT CustomerID, FirstName FROM Customers"
    customer_id_df = pd.read_sql(customer_id_query, order_management_connection)
    # Merge customer_id_df with merged_customer_df on the firstname column
    final_customers_merged_df = pd.merge(customers_merged_df, customer_id_df, how='left', on='FirstName')
    final_customers_merged_df.drop_duplicates(inplace=True)
except Exception as e:
    print(f"Error in fetching CustomerID data from database: {e}")

final_customers_merged_df.head()

,OrderID,FirstName,CustomerID
0,10107,Land of Toys Inc.,30
1,10121,Reims Collectables,31
2,10134,Lyon Souveniers,32
3,10145,Toys4GrownUps.com,33
4,10159,Corporate Gift Ideas Co.,34


In [22]:
try:
    # Assign the customerid column from final_customers_merged_df to csv_orders_df
    csv_orders_df.insert(1, 'CustomerID', final_customers_merged_df['CustomerID'])
except Exception as e:
    print(f"Error moving CustomerID column: {e}")

csv_orders_df.head()


,OrderID,CustomerID,OrderDate,ShippingAddress,Status
0,10107,30.0,2003-02-24,NaN,Shipped
1,10121,31.0,2003-05-07,NaN,Shipped
2,10134,32.0,2003-07-01,NaN,Shipped
3,10145,33.0,2003-08-25,NaN,Shipped
4,10159,34.0,2003-10-10,NaN,Shipped


In [23]:
# Load the orders data into the order_management database
try:
    csv_orders_df.to_sql('Orders', order_management_engine, if_exists='append', index=False)
except Exception as e:
    print(f"Error loading data into the order_management database: {e}")

In [24]:
# Extract and transform order details (items) data from the CSV file
try:
    csv_order_items_df = csv_sales_data_df[['ORDERNUMBER', 'QUANTITYORDERED', 'PRICEEACH']].copy()
    # Modify the columns from the CSV file to fit the order_management database
    csv_order_items_df.rename(columns={
        'ORDERNUMBER': 'OrderID',
        'QUANTITYORDERED': 'Quantity',
        'PRICEEACH': 'Price'
    }, inplace=True)
except Exception as e:
    print(f"Error extracting and transforming order items data: {e}")

csv_order_items_df.head()

,OrderID,Quantity,Price
0,10107,30,95.70
1,10121,34,81.35
2,10134,41,94.74
3,10145,45,83.26
4,10159,49,100.00


In [25]:
# Get the productid for each order item by merging dataframes
try:
    # Merge the csv_order_items_df with csv_sales_data_df on the orderid and ordernumber columns
    csv_orders_merged_df = pd.merge(csv_order_items_df, csv_sales_data_df, how='left', left_on='OrderID', right_on='ORDERNUMBER')
    csv_orders_merged_df.drop_duplicates(inplace=True)
    csv_orders_merged_df.drop('ORDERNUMBER', axis=1, inplace=True)
    # Extract the orderid and productline columns from csv_orders_merged_df
    order_product_df = csv_orders_merged_df[['OrderID', 'PRODUCTLINE']].copy()
    # Merge the order_product_df with csv_products_df on the name column
    products_merged_df = pd.merge(order_product_df, csv_products_df[['Name']], how='left', left_on='PRODUCTLINE', right_on='Name')
    products_merged_df.drop('PRODUCTLINE', axis=1, inplace=True)
    products_merged_df.drop_duplicates(inplace=True)
except Exception as e:
    print(f"Error in merging and processing dataframes: {e}")

try:
    # SQL query to fetch productid and name from products table
    product_id_query = "SELECT ProductID, Name FROM Products"
    product_id_df = pd.read_sql(product_id_query, order_management_connection)
    # Merge product_id_df with products_merged_df on the name column
    final_products_merged_df = pd.merge(products_merged_df, product_id_df, how='left', on='Name')
    final_products_merged_df.drop_duplicates(inplace=True)
except Exception as e:
    print(f"Error in fetching ProductID data from database: {e}")

final_products_merged_df.head()

,OrderID,Name,ProductID
0,10107,Motorcycles,100
1,10107,Motorcycles,101
2,10107,Motorcycles,102
3,10107,Motorcycles,103
4,10107,Motorcycles,104


In [26]:
try:
    # Assign the productid column from final_products_merged_df to csv_order_items_df
    csv_order_items_df.insert(1, 'ProductID', final_products_merged_df['ProductID'])
except Exception as e:
    print(f"Error assigning ProductID column: {e}")

csv_order_items_df.head()

,OrderID,ProductID,Quantity,Price
0,10107,100,30,95.70
1,10121,101,34,81.35
2,10134,102,41,94.74
3,10145,103,45,83.26
4,10159,104,49,100.00


In [27]:
# Load the order details (items) data into the order_management database
try:
    csv_order_items_df.to_sql('OrderItems', order_management_engine, if_exists='append', index=False)
except Exception as e:
    print(f"Error loading data into the order_management database: {e}")

In [28]:
# Extract and transform suppliers data from the CSV file
try:
    csv_suppliers_df = csv_sales_data_df[['CONTACTFIRSTNAME', 'CONTACTLASTNAME', 'PHONE', 'ADDRESSLINE1']].copy()
    csv_suppliers_df.drop_duplicates(inplace=True)
    # Modify the columns from the CSV file to fit the order_management database
    csv_suppliers_df.rename(columns={
        'CONTACTFIRSTNAME': 'FirstName',
        'CONTACTLASTNAME': 'LastName',
        'PHONE': 'Phone',
        'ADDRESSLINE1': 'Address'
    }, inplace=True)
    # Fill the company column with mock supplier data
    csv_suppliers_df['Company'] = ['Supplier ' + random.choice(string.ascii_uppercase) for letter in range(len(csv_suppliers_df))]
except Exception as e:
    print(f"Error extracting and transforming supplier data: {e}")

csv_suppliers_df.head()

,FirstName,LastName,Phone,Address,Company
0,Kwai,Yu,2125557818,897 Long Airport Avenue,Supplier J
1,Paul,Henriot,26.47.1555,59 rue de l'Abbaye,Supplier T
2,Daniel,Da Cunha,+33 1 46 62 7555,27 rue du Colonel Pierre Avia,Supplier H
3,Julie,Young,6265557265,78934 Hillside Dr.,Supplier K
4,Julie,Brown,6505551386,7734 Strong St.,Supplier O


In [29]:
# Load the suppliers data into the order_management database
try:
    csv_suppliers_df.to_sql('Suppliers', order_management_engine, if_exists='append', index=False)
except Exception as e:
    print(f"Error loading data into the order_management database: {e}")

### SECTION 5: EXTRACT DATA FROM AN API CALL AND LOAD DATA INTO THE ORDER_MANAGEMENT DATABASE

In [30]:
# Define the URL for the Fake Store API's products endpoint
api_url = 'https://fakestoreapi.com/products'

# Send a GET request to the API to retrieve product data
response = requests.get(api_url)

In [31]:
# Check if the request was successful
if response.status_code == 200:
    try:
        # Convert the JSON response to a DataFrame
        api_products_data = response.json()
        api_products_df = pd.DataFrame(api_products_data)
        # Extract and transform products data from the API call
        api_products_df = api_products_df[['title', 'description', 'price', 'rating']]
        api_products_df['StockQuantity'] = api_products_df['rating'].apply(lambda x: x['count'])
        api_products_df.drop(columns=['rating'], inplace=True)
        # Modify the columns from the API call to fit the order_management database
        api_products_df.rename(columns={
            'title': 'Name',
            'description': 'Description',
            'price': 'Price',
        }, inplace=True)
    except Exception as e:
        print(f"Error converting API response to DataFrame: {e}")

api_products_df.head()

,Name,Description,Price,StockQuantity
0,"Fjallraven - Foldsack No. 1 Backpack, Fits 15 ...",Your perfect pack for everyday use and walks i...,109.95,120
1,Mens Casual Premium Slim Fit T-Shirts,"Slim-fitting style, contrast raglan long sleev...",22.30,259
2,Mens Cotton Jacket,great outerwear jackets for Spring/Autumn/Wint...,55.99,500
3,Mens Casual Slim Fit,The color could be slightly different between ...,15.99,430
4,John Hardy Women's Legends Naga Gold & Silver ...,"From our Legends Collection, the Naga was insp...",695.00,400


In [32]:
# Load the products data into the order_management database
try:
    api_products_df.to_sql('Products', order_management_engine, if_exists='append', index=False)
except Exception as e:
    print(f"Error loading data into the order_management database: {e}")

### SECTION 6: AUTHOR SQL QUERIES (SELECT STATEMENTS) TO DEMONSTRATE PROPER DATABASE FUNCTIONALITY

In [33]:
# Connect to the order_management database
try:
    sql_queries_connection = pymysql.connect(host=host, user=user, password=password, database=main_database)
    print("Connection to the order_management database was successful!")
except Exception as e:
    print(f"Error connecting to the order_management database: {e}")

Connection to the order_management database was successful!


In [34]:
# Query 1: Retrieve Customer Information with Total Order Amounts
df_customer_sales = pd.read_sql("""
    SELECT
        c.CustomerID,
        c.FirstName,
        c.LastName,
        SUM(od.Quantity * p.Price) AS TotalAmount
    FROM
        Customers c
        JOIN Orders o ON c.CustomerID = o.CustomerID
        JOIN OrderItems od ON o.OrderID = od.OrderID
        JOIN Products p ON od.ProductID = p.ProductID
    GROUP BY
        c.CustomerID,
        c.FirstName,
        c.LastName
    ORDER BY
        TotalAmount DESC;
    """, sql_queries_connection)

print("Query 1: Customer Information with Top 10 Total Order Amounts")
df_customer_sales.head(10)

Query 1: Customer Information with Top 10 Total Order Amounts


,CustomerID,FirstName,LastName,TotalAmount
0,53,Euro Shopping Channel,,541778.0
1,69,Mini Gifts Distributors Ltd.,,366376.0
2,40,"Australian Collectors, Co.",,179029.0
3,78,Danish Wholesale Imports,,178234.0
4,56,"Dragon Souveniers, Ltd.",,171543.0
5,30,Land of Toys Inc.,,154874.0
6,60,"Saveley & Henriot, Co.",,148492.0
7,50,Souveniers And Things Co.,,143119.0
8,103,Suominen Souveniers,,141726.0
9,99,"Vida Sport, Ltd",,134483.0


In [35]:
# Query 2: Calculate Average Order Amount by Customer
df_average_order_amount = pd.read_sql("""
    SELECT
        c.CustomerID,
        c.FirstName,
        c.LastName,
        AVG(od.Quantity * p.Price) AS AverageOrderAmount
    FROM
        Customers c
        JOIN Orders o ON c.CustomerID = o.CustomerID
        JOIN OrderItems od ON o.OrderID = od.OrderID
        JOIN Products p ON od.ProductID = p.ProductID
    GROUP BY
        c.CustomerID,
        c.FirstName,
        c.LastName
    ORDER BY
        AverageOrderAmount DESC;
    """, sql_queries_connection)

print("\nQuery 2: Top 10 Average Order Amounts by Customer")
df_average_order_amount.head(10)


Query 2: Top 10 Average Order Amounts by Customer


,CustomerID,FirstName,LastName,AverageOrderAmount
0,7,Ming-Yang,Xie,13800.000000
1,88,Gifts4AllAges.com,,9071.000000
2,82,"Double Decker Gift Stores, Ltd",,6013.000000
3,79,"Stylish Desk Decors, Co.",,4907.666667
4,80,Mini Creations Ltd.,,4307.500000
5,114,Signal Gift Stores,,4229.750000
6,65,"Tokyo Collectables, Ltd",,4222.750000
7,120,Royale Belge,,4218.125000
8,74,Osaka Souveniers Co.,,4136.533333
9,86,Petit Auto,,4134.000000


In [36]:
# Query 3: Retrieve Product Information with Total Sales
df_product_sales = pd.read_sql("""
    SELECT
        p.ProductID,
        p.Name AS ProductName,
        SUM(od.Quantity) AS TotalUnitsSold,
        SUM(od.Quantity * p.Price) AS TotalSalesAmount
    FROM
        Products p
        JOIN OrderItems od ON p.ProductID = od.ProductID
    GROUP BY
        p.ProductID,
        p.Name
    ORDER BY
        TotalSalesAmount DESC;
    """, sql_queries_connection)

print("\nQuery 3: Product Information with Top 10 Total Sales")
df_product_sales.head(10)


Query 3: Product Information with Top 10 Total Sales


,ProductID,ProductName,TotalUnitsSold,TotalSalesAmount
0,162,Motorcycles,261.0,50373.0
1,165,Motorcycles,234.0,45162.0
2,160,Motorcycles,232.0,44776.0
3,177,Motorcycles,232.0,44776.0
4,167,Motorcycles,229.0,44197.0
5,179,Motorcycles,225.0,43425.0
6,175,Motorcycles,220.0,42460.0
7,176,Motorcycles,216.0,41688.0
8,163,Motorcycles,212.0,40916.0
9,172,Motorcycles,212.0,40916.0
